# 4.7. Regresja liniowa wielu zmiennych

Rozpoczęliśmy swoje rozważania nad regresją liniową od przewidywania jakości wina tylko na podstawie zawartości alkoholu. Mamy jednak kilka innych zmiennych, które mogą być dość wartościowe. Spróbujmy więc dołożyć wszystkie pozostałe cechy, które zgromadziliśmy i zobaczmy jak wpływa to na skuteczność predykcji. Regresja w dwóch wymiarach jest atrakcyjna z jednego powodu - łatwo jest nam zwizualizować stworzoną funkcję, co nie jest już wykonalne w wyżejwymiarowych przestrzeniach.

In [1]:
import pandas as pd

In [2]:
wine_quality_df = pd.read_parquet("./data/wine-quality.parquet")
wine_quality_df.sample(n=5)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,color_red,color_white,bound sulfur dioxide
4066,7.1,0.440,0.27,8.4,0.057,60.0,160.0,0.99257,3.16,0.36,11.8,6,0,1,100.0
730,9.5,0.550,0.66,2.3,0.387,12.0,37.0,0.99820,3.17,0.67,9.6,5,1,0,25.0
3723,5.7,0.265,0.28,6.9,0.036,46.0,150.0,0.99299,3.36,0.44,10.8,7,0,1,104.0
3707,6.7,0.340,0.43,1.6,0.041,29.0,114.0,0.99014,3.23,0.44,12.6,6,0,1,85.0
4195,7.1,0.450,0.24,2.7,0.040,24.0,87.0,0.98862,2.94,0.38,13.4,8,0,1,63.0


Do stworzenia tego modelu wykorzystamy wszystkie omówione wcześniej techniki. Po raz pierwszy skorzystamy również z mechanizmu tzw. *pipelines* dostarczanych przez scikit-learn.

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
train_df, test_df = train_test_split(wine_quality_df, test_size=0.2)

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

In [11]:
pipeline = Pipeline(steps=[
    ("scaler", StandardScaler()),
    ("regressor", LinearRegression()),
])

In [13]:
pipeline.fit(train_df.drop("quality", axis="columns"), 
             train_df["quality"])

Pipeline(memory=None,
         steps=[('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('regressor',
                 LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
                                  normalize=False))],
         verbose=False)

In [17]:
train_pred = pipeline.predict(train_df.drop("quality", 
                                            axis="columns"))
test_pred = pipeline.predict(test_df.drop("quality", 
                                          axis="columns"))

In [18]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [20]:
mean_absolute_error(train_df["quality"], train_pred), \
    mean_absolute_error(test_df["quality"], test_pred)

(0.5691584019624648, 0.5686292410727408)

In [21]:
mean_squared_error(train_df["quality"], train_pred), \
    mean_squared_error(test_df["quality"], test_pred)

(0.5382273452404189, 0.5296936519106971)

In [29]:
pipeline.steps[1][1].coef_, pipeline.steps[1][1].intercept_

(array([ 0.10558157, -0.23635444, -0.00847628,  0.28972962, -0.02900862,
         0.06232756, -0.01935634, -0.30312757,  0.07523227,  0.11067352,
         0.26189113,  0.07284289, -0.07284289, -0.04810621]),
 5.822974793149908)

In [28]:
train_df.drop("quality", axis="columns").columns

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'color_red', 'color_white',
       'bound sulfur dioxide'],
      dtype='object')

In [32]:
feature_weights = dict(
    zip(
        train_df.drop("quality", axis="columns").columns,
        pipeline.steps[1][1].coef_
    )
)
feature_weights

{'fixed acidity': 0.10558157362894088,
 'volatile acidity': -0.23635443645529206,
 'citric acid': -0.008476283622600432,
 'residual sugar': 0.28972961551167686,
 'chlorides': -0.02900861620212126,
 'free sulfur dioxide': 0.062327556387413066,
 'total sulfur dioxide': -0.01935633646894997,
 'density': -0.30312756644283456,
 'pH': 0.07523227277980737,
 'sulphates': 0.11067352450605676,
 'alcohol': 0.26189113268616315,
 'color_red': 0.07284289253219821,
 'color_white': -0.07284289253219826,
 'bound sulfur dioxide': -0.04810620636219985}